In [1]:
from __future__ import print_function
from sympy import *
from eqn_manip import move_terms, move_terms_left, divide_terms, mult_eqn, get_coeff_for
from IPython.display import display
init_printing()

### Cubic spline equations
First step is make the a straightforward derivation of some cubic spline equations.  These could be further simplified to a tridiagonal matrix and efficiently solved.  For small numbers of knots and for testing purposes, it is sufficient to use a larger matrix and a general solver.

The first derivation will involve knots with constant separation of 1.  Later the derivation will be repeated with arbitrary knot positions.

In [2]:
# Distance from the previous knot, for the case of uniform knot spacing
t = Symbol('t')

# Number of knots
n = Symbol('n', integer=True)
i = Symbol('i', integer=True)
# Function values to intepolated at the knots
y = IndexedBase('y')

# Coefficients of the spline function
a,b,c,d = [IndexedBase(s) for s in 'a b c d'.split()]

# Cubic spline function
s = a + b*t + c*t*t + d*t**3
display(Eq(y,s))

# With indexed variables
si = a[i] + b[i]*t + c[i]*t*t + d[i]*t**3
display(Eq(y[i],si))

     3      2            
y = t ⋅d + t ⋅c + t⋅b + a

        3         2                     
y[i] = t ⋅d[i] + t ⋅c[i] + t⋅b[i] + a[i]

### Set up equations

In [3]:
# Value at knots (t=0)
eq1 = Eq(si.subs(t,0), y[i])
eq1

a[i] = y[i]

In [4]:
# Value at knots (t=1)
eq2 = Eq(si.subs(t,1), y[i+1])
eq2

a[i] + b[i] + c[i] + d[i] = y[i + 1]

In [5]:
# Continuity of first derivatives at knots 
dsi = diff(si, t)
eq3 = Eq(dsi.subs(t,1), dsi.subs(i, i+1).subs(t,0))
eq3

b[i] + 2⋅c[i] + 3⋅d[i] = b[i + 1]

In [6]:
# Continuity of second derivatives at knots
d2si = diff(si, t, 2)
eq4 = Eq(d2si.subs(t,1), d2si.subs(i, i+1).subs(t,0))
eq4

2⋅c[i] + 6⋅d[i] = 2⋅c[i + 1]

### Boundary conditions

In [7]:
# Natural BC (second derivative is zero at both ends)
eq5 = Eq(d2si.subs(i,0).subs(t,0), 0)
display(eq5)
eq6 = Eq(d2si.subs(i,n-1).subs(t,1), 0)
eq6

2⋅c[0] = 0

2⋅c[n - 1] + 6⋅d[n - 1] = 0

In [8]:
# Or
# Specified first derivatives at boundaries
eq5b = Eq(dsi.subs(i,0).subs(t,0),0)
display(eq5b)
eq6b = Eq(dsi.subs(i,n-1).subs(t,1),0)
eq6b

b[0] = 0

b[n - 1] + 2⋅c[n - 1] + 3⋅d[n - 1] = 0

### Set up equations

There are 4$n$ unknowns.  To get the right number of equations, we need 
 - one of eq5 and eq6 (boundary conditions) (2 equations)
 - eq1 and eq2 (values at beginning and end of each interval) for every knot ($2n$ equations)
 - eq3 and eq4 (continuity of 1st and 2nd derivatives) for every interior knot ($2(n-1)$ equations)

This gives the necessary $4n$ equations

In [9]:
display(eq1)
display(eq2)
display(eq3)
display(eq4)

a[i] = y[i]

a[i] + b[i] + c[i] + d[i] = y[i + 1]

b[i] + 2⋅c[i] + 3⋅d[i] = b[i + 1]

2⋅c[i] + 6⋅d[i] = 2⋅c[i + 1]

In [10]:
# Equations 3 and 4 need to be rearranged so all the unknowns are the left-hand side
sym_lhs = [a[i],b[i],c[i],d[i],a[i+1],b[i+1],c[i+1],d[i+1]]
sym_rhs = [y[i]]
eq3b = divide_terms(eq3, sym_lhs, sym_rhs)
display(eq3b)
eq4b = divide_terms(eq4, sym_lhs, sym_rhs)
display(eq4b)

-b[i + 1] + b[i] + 2⋅c[i] + 3⋅d[i] = 0

-2⋅c[i + 1] + 2⋅c[i] + 6⋅d[i] = 0

In [11]:
# Set up matrix with concrete number of knots
nknots = 3
nval = nknots -1
neqn = 4*nval
m = Matrix.eye(neqn, neqn)
rhs = Matrix.eye(neqn,1)
unknowns = Matrix.eye(neqn,1)

In [12]:
use_natural_bc = True
symlist = list()
for idx in range(nval):
    symlist.append(a[idx])
    symlist.append(b[idx])
    symlist.append(c[idx])
    symlist.append(d[idx])
    
for idx,sym in enumerate(symlist):
    unknowns[idx] = sym

# First rows
for idx,sym in enumerate(symlist):
    if use_natural_bc:    
        m[0,idx] = get_coeff_for(eq5.lhs.subs(i,0), sym , symlist)
    else:
        m[0,idx] = get_coeff_for(eq5b.lhs.subs(i,0), sym, symlist)
    m[1,idx] = get_coeff_for(eq1.lhs.subs(i,0), sym,symlist)
    m[2,idx] = get_coeff_for(eq2.lhs.subs(i,0), sym,symlist)
    
rhs[0] = eq5.rhs.subs(i,0)
rhs[1] = eq1.rhs.subs(i,0)
rhs[2] = eq2.rhs.subs(i,0)

jdx = 3
for nv in range(1,nval):
    for idx,sym in enumerate(symlist):
        m[jdx+0,idx] = get_coeff_for(eq1.lhs.subs(i,nv), sym,symlist)
        m[jdx+1,idx] = get_coeff_for(eq2.lhs.subs(i,nv), sym,symlist)
        m[jdx+2,idx] = get_coeff_for(eq3b.lhs.subs(i,nv-1), sym,symlist)
        m[jdx+3,idx] = get_coeff_for(eq4b.lhs.subs(i,nv-1), sym,symlist)
    rhs[jdx+0] = eq1.rhs.subs(i,nv)
    rhs[jdx+1] = eq2.rhs.subs(i,nv)
    rhs[jdx+2] = eq3b.rhs.subs(i,nv)
    rhs[jdx+3] = eq4b.rhs.subs(i,nv)
    jdx += 4

# Last rows
for idx,sym in enumerate(symlist):
    if use_natural_bc:
        m[jdx,idx] = get_coeff_for(eq6.lhs.subs(n,nval), sym , symlist)
    else:
        m[jdx,idx] = get_coeff_for(eq6b.lhs.subs(n,nval), sym , symlist)
rhs[jdx] = eq6.rhs.subs(n,nval)

# Display of matrix-vector multiplication is not working - pretend the first equals is a multiplication
with evaluate(False):
    display(Eq(Eq(m,unknowns),rhs))

⎡0  0  2  0  0  0   0   0⎤   ⎡a[0]⎤   ⎡ 0  ⎤
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎢1  0  0  0  0  0   0   0⎥   ⎢b[0]⎥   ⎢y[0]⎥
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎢1  1  1  1  0  0   0   0⎥   ⎢c[0]⎥   ⎢y[1]⎥
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎢0  0  0  0  1  0   0   0⎥   ⎢d[0]⎥   ⎢y[1]⎥
⎢                        ⎥ = ⎢    ⎥ = ⎢    ⎥
⎢0  0  0  0  1  1   1   1⎥   ⎢a[1]⎥   ⎢y[2]⎥
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎢0  1  2  3  0  -1  0   0⎥   ⎢b[1]⎥   ⎢ 0  ⎥
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎢0  0  2  6  0  0   -2  0⎥   ⎢c[1]⎥   ⎢ 0  ⎥
⎢                        ⎥   ⎢    ⎥   ⎢    ⎥
⎣0  0  0  0  0  0   2   6⎦   ⎣d[1]⎦   ⎣ 0  ⎦

In [13]:
# Solve
sln = m.LUsolve(rhs)
sln

⎡          y[0]          ⎤
⎢                        ⎥
⎢  5⋅y[0]   3⋅y[1]   y[2]⎥
⎢- ────── + ────── - ────⎥
⎢    4        2       4  ⎥
⎢                        ⎥
⎢           0            ⎥
⎢                        ⎥
⎢   y[0]   y[1]   y[2]   ⎥
⎢   ──── - ──── + ────   ⎥
⎢    4      2      4     ⎥
⎢                        ⎥
⎢          y[1]          ⎥
⎢                        ⎥
⎢       y[0]   y[2]      ⎥
⎢     - ──── + ────      ⎥
⎢        2      2        ⎥
⎢                        ⎥
⎢3⋅y[0]   3⋅y[1]   3⋅y[2]⎥
⎢────── - ────── + ──────⎥
⎢  4        2        4   ⎥
⎢                        ⎥
⎢    y[0]   y[1]   y[2]  ⎥
⎢  - ──── + ──── - ────  ⎥
⎣     4      2      4    ⎦

In [14]:
# Can substitute concrete knot values
ysubs = {y[0]:0.5, y[1]:1.1, y[2]:1.5}
ysubs

{y[0]: 0.5, y[1]: 1.1, y[2]: 1.5}

In [15]:
# Create substitutions for spline coefficients
subslist = dict(zip(symlist,sln))
subslist

⎧                                5⋅y[0]   3⋅y[1]   y[2]          y[0]   y[2]  
⎨a[0]: y[0], a[1]: y[1], b[0]: - ────── + ────── - ────, b[1]: - ──── + ────, 
⎩                                  4        2       4             2      2    

               3⋅y[0]   3⋅y[1]   3⋅y[2]        y[0]   y[1]   y[2]          y[0
c[0]: 0, c[1]: ────── - ────── + ──────, d[0]: ──── - ──── + ────, d[1]: - ───
                 4        2        4            4      2      4             4 

]   y[1]   y[2]⎫
─ + ──── - ────⎬
     2      4  ⎭

In [16]:
# For a particular interval
si0 = si.subs(i,0).subs(subslist)
display(si0)
si0b = si0.subs(ysubs)
display(si0b)

 3 ⎛y[0]   y[1]   y[2]⎞     ⎛  5⋅y[0]   3⋅y[1]   y[2]⎞       
t ⋅⎜──── - ──── + ────⎟ + t⋅⎜- ────── + ────── - ────⎟ + y[0]
   ⎝ 4      2      4  ⎠     ⎝    4        2       4  ⎠       

        3               
- 0.05⋅t  + 0.65⋅t + 0.5

In [17]:
# Can take the seconnd derivative, to compare with the output values from some spline solvers
# At t=0, values is zero, as expected from natural boundary conditions
display(diff(si0b, t, 2).subs(t,0))
# Compute at t=1
display(diff(si0b, t, 2).subs(t,1))

0

-0.300000000000000

# Derive for general knot spacing

In [18]:
# Distance from the previous knot
# t[i] ranges from 0 to x[i+1]-x[i]
t = IndexedBase('t')

# Number of knots
n = Symbol('n', integer=True)
i = Symbol('i', integer=True)
# Location of knots
x = IndexedBase('x')
# Function values to intepolated at the knots
y = IndexedBase('y')

# Coefficients of the spline function
a,b,c,d = [IndexedBase(s) for s in 'a b c d'.split()]


# With indexed variables
si = a[i] + b[i]*t[i] + c[i]*t[i]*t[i] + d[i]*t[i]**3
display(Eq(y[i],si))

                                   2            3
y[i] = a[i] + b[i]⋅t[i] + c[i]⋅t[i]  + d[i]⋅t[i] 

In [19]:
# Value at knots (t[i]=0)
eq1 = Eq(si.subs(t[i],0), y[i])
eq1

a[i] = y[i]

In [20]:
# Value at knots (t[i]=x[i+1])
eq2 = Eq(si.subs(t[i],x[i+1]-x[i]), y[i+1])
eq2

                 3                         2                                  
(x[i + 1] - x[i]) ⋅d[i] + (x[i + 1] - x[i]) ⋅c[i] + (x[i + 1] - x[i])⋅b[i] + a

              
[i] = y[i + 1]

In [21]:
# Continuity of first derivatives at knots 
dsi = diff(si, t[i])
eq3 = Eq(dsi.subs(t[i],x[i+1]-x[i]), dsi.subs(i, i+1).subs(t[i+1],0))
eq3

                   2                                                  
3⋅(x[i + 1] - x[i]) ⋅d[i] + 2⋅(x[i + 1] - x[i])⋅c[i] + b[i] = b[i + 1]

In [22]:
# Continuity of second derivatives at knots
d2si = diff(si, t[i], 2)
eq4 = Eq(d2si.subs(t[i],x[i+1]-x[i]), d2si.subs(i, i+1).subs(t[i+1],0))
eq4

6⋅(x[i + 1] - x[i])⋅d[i] + 2⋅c[i] = 2⋅c[i + 1]

In [23]:
# Natural BC (second derivative is zero at both ends)
eq5 = Eq(d2si.subs(i,0).subs(t[0],0), 0)
display(eq5)
eq6 = Eq(d2si.subs(t[i],x[i+1]-x[i]).subs(i,n-1), 0)
eq6

2⋅c[0] = 0

6⋅(-x[n - 1] + x[n])⋅d[n - 1] + 2⋅c[n - 1] = 0

In [24]:
# Or
# Specified first derivatives at boundaries
eq5b = Eq(dsi.subs(i,0).subs(t[0],0),0)
display(eq5b)
eq6b = Eq(dsi.subs(t[i],x[i+1]-x[i]).subs(i,n-1),0)
eq6b

b[0] = 0

                    2                                                        
3⋅(-x[n - 1] + x[n]) ⋅d[n - 1] + 2⋅(-x[n - 1] + x[n])⋅c[n - 1] + b[n - 1] = 0

In [25]:
# Equations 3 and 4 need to be rearranged so all the unknowns are the left-hand side
sym_lhs = [a[i],b[i],c[i],d[i],a[i+1],b[i+1],c[i+1],d[i+1]]
sym_rhs = [y[i]]
eq3b = divide_terms(eq3, sym_lhs, sym_rhs)
display(eq3b)
eq4b = divide_terms(eq4, sym_lhs, sym_rhs)
display(eq4b)

                   2                                                      
3⋅(x[i + 1] - x[i]) ⋅d[i] + 2⋅(x[i + 1] - x[i])⋅c[i] - b[i + 1] + b[i] = 0

6⋅(x[i + 1] - x[i])⋅d[i] - 2⋅c[i + 1] + 2⋅c[i] = 0

In [26]:
display(eq1)
display(eq2)
display(eq3b)
display(eq4b)

a[i] = y[i]

                 3                         2                                  
(x[i + 1] - x[i]) ⋅d[i] + (x[i + 1] - x[i]) ⋅c[i] + (x[i + 1] - x[i])⋅b[i] + a

              
[i] = y[i + 1]

                   2                                                      
3⋅(x[i + 1] - x[i]) ⋅d[i] + 2⋅(x[i + 1] - x[i])⋅c[i] - b[i + 1] + b[i] = 0

6⋅(x[i + 1] - x[i])⋅d[i] - 2⋅c[i + 1] + 2⋅c[i] = 0

In [27]:
# Set up matrix with concrete number of knots
nval = 2
nknots = nval + 1
neqn = 4*nval
m = Matrix.eye(neqn, neqn)
rhs = Matrix.eye(neqn,1)
unknowns = Matrix.eye(neqn,1)

In [28]:
use_natural_bc = True
symlist = list()
for idx in range(nval):
    symlist.append(a[idx])
    symlist.append(b[idx])
    symlist.append(c[idx])
    symlist.append(d[idx])
    
for idx,sym in enumerate(symlist):
    unknowns[idx] = sym

# First rows
for idx,sym in enumerate(symlist):
    if use_natural_bc:    
        m[0,idx] = get_coeff_for(eq5.lhs.subs(i,0), sym , symlist)
    else:
        m[0,idx] = get_coeff_for(eq5b.lhs.subs(i,0), sym, symlist)
    m[1,idx] = get_coeff_for(eq1.lhs.subs(i,0), sym,symlist)
    m[2,idx] = get_coeff_for(eq2.lhs.subs(i,0), sym,symlist)
    
rhs[0] = eq5.rhs.subs(i,0)
rhs[1] = eq1.rhs.subs(i,0)
rhs[2] = eq2.rhs.subs(i,0)

jdx = 3
for nv in range(1,nval):
    for idx,sym in enumerate(symlist):
        m[jdx+0,idx] = get_coeff_for(eq1.lhs.subs(i,nv), sym,symlist)
        m[jdx+1,idx] = get_coeff_for(eq2.lhs.subs(i,nv), sym,symlist)
        m[jdx+2,idx] = get_coeff_for(eq3b.lhs.subs(i,nv-1), sym,symlist)
        m[jdx+3,idx] = get_coeff_for(eq4b.lhs.subs(i,nv-1), sym,symlist)
    rhs[jdx+0] = eq1.rhs.subs(i,nv)
    rhs[jdx+1] = eq2.rhs.subs(i,nv)
    rhs[jdx+2] = eq3b.rhs.subs(i,nv)
    rhs[jdx+3] = eq4b.rhs.subs(i,nv)
    jdx += 4

# Last rows
for idx,sym in enumerate(symlist):
    if use_natural_bc:
        m[jdx,idx] = get_coeff_for(eq6.lhs.subs(n,nval), sym , symlist)
    else:
        m[jdx,idx] = get_coeff_for(eq6b.lhs.subs(n,nval), sym , symlist)
rhs[jdx] = eq6.rhs.subs(n,nval)

# Display just the matrix, displaying other pieces doesn't work, and they are the same as previously
m

⎡0       0               2                  0          0       0              
⎢                                                                             
⎢1       0               0                  0          0       0              
⎢                                                                             
⎢                               2                  3                          
⎢1  -x[0] + x[1]  (-x[0] + x[1])     (-x[0] + x[1])    0       0              
⎢                                                                             
⎢0       0               0                  0          1       0              
⎢                                                                             
⎢                                                                             
⎢0       0               0                  0          1  -x[1] + x[2]  (-x[1]
⎢                                                                             
⎢                                                   

In [29]:
# Solve
sln = m.LUsolve(rhs)
# Display is large - skip for now, uncomment if you want to see it
#sln

In [30]:
# Create substitutions for spline coefficients
subslist = dict(zip(symlist,sln))
#subslist

In [31]:
# Can substitute concrete knot values
ysubs = {y[0]:0.5, y[1]:1.1, y[2]:1.5}
display(ysubs)
# Use uniform knots to verify answer is the same as before
xsubs = {x[0]:0.0, x[1]:1.0, x[2]:2.0}
display(xsubs)

{y[0]: 0.5, y[1]: 1.1, y[2]: 1.5}

{x[0]: 0.0, x[1]: 1.0, x[2]: 2.0}

In [32]:
# For a particular interval
si0 = si.subs(i,0).subs(subslist)
#display(si0)
si0b = si0.subs(ysubs).subs(xsubs).subs(t[0],t)
display(si0b)

        3               
- 0.05⋅t  + 0.65⋅t + 0.5